In [1]:
import pickle

# load model

model = pickle.load(open('svc_model.pkl','rb'))

In [2]:
model

SVC(probability=True)

In [5]:
import python_speech_features as mfcc
import os
from scipy.io import wavfile
#from hmmlearn import hmm
import numpy as np
import pandas as pd
import librosa

In [6]:
train_csv = pd.read_csv('preprocessed/train.csv')
mfcc_features = pd.read_csv('mfcc_features.csv')

In [7]:
mfcc_features['accent'].value_counts()

España             74992
México             15448
Andino             11699
Rioplatense        10621
Caribe              7661
América central     5565
Chileno             3918
Name: accent, dtype: int64

In [8]:
"""replacement = {'México':'Latinoamericano',
               'América central':'Latinoamericano',
               'Andino':'Latinoamericano',
               'Rioplatense':'Latinoamericano',
               'Chileno': 'Latinoamericano',
               'Caribe': 'Latinoamericano',
              }
"""

replacement = {'México':0,
               'América central':0,
               'Andino':0,
               'Rioplatense':0,
               'Chileno': 0,
               'Caribe': 0,
               'España':1
              }

mfcc_features['accent'] = mfcc_features['accent'].replace(replacement)


In [10]:
mfcc_features['accent'].value_counts()

1    74992
0    54912
Name: accent, dtype: int64

In [11]:
mfcc_features['accent'][:5000].value_counts()

0    3656
1    1344
Name: accent, dtype: int64

In [72]:
from sklearn.preprocessing import StandardScaler

def scale_features(data):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(np.array(data.iloc[:, 1:], dtype = float))
    # with data.iloc[:, 0:-1] we don't consider the label column
        
    return scaled_data, scaler


x, scaler = scale_features(mfcc_features)

In [73]:
from sklearn.preprocessing import LabelEncoder

def get_labels(data):
    labels = data.iloc[:, 1]
    encoder = LabelEncoder()
    labels = encoder.fit_transform(labels)
    return labels, encoder

y, encoder = get_labels(mfcc_features)
classes = encoder.classes_
print("Before encoding:", mfcc_features.iloc[0].values[1])
print("\nAfter encoding:", y[1])
print("\nClasses:", classes)

Before encoding: 0.0

After encoding: 0

Classes: [0 1]


In [46]:
 mfcc_features.iloc[129899]

Unnamed: 0            129899.000000
accent                     2.000000
spectral_centroid       1322.034468
spectral_bandwidth      1492.653932
spectral_rolloff        2617.811366
mfcc1                   -492.601620
mfcc2                    118.546074
mfcc3                     13.053954
mfcc4                     42.518433
mfcc5                     39.099270
mfcc6                     12.175418
mfcc7                     22.940990
mfcc8                      4.345521
mfcc9                    -12.084730
mfcc10                     6.653798
mfcc11                     5.192945
mfcc12                    -7.270759
mfcc13                    -7.877683
Name: 129899, dtype: float64

In [60]:
mfcc_features[:10000]

,Unnamed: 0,accent,spectral_centroid,spectral_bandwidth,spectral_rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,0,0,1677.718785,1612.745697,3358.794145,-569.73413,128.704030,6.969439,8.466482,9.177470,2.072889,3.930073,-15.301613,-7.267760,10.540461,-2.708580,-6.161283,-2.961534
1,1,0,1619.961935,1550.527875,3185.866592,-536.89484,132.884670,14.828790,4.359267,3.613036,5.550890,5.075633,-21.225660,-12.947938,6.962500,-4.414804,-6.305440,-3.568841
2,2,0,1681.779715,1537.107681,3294.990647,-552.82900,133.168930,11.685370,-2.057885,-0.008511,1.975699,4.581186,-15.395591,-10.483634,6.167295,-1.988072,-3.231391,-0.737840
3,3,0,1845.987325,1679.445156,3671.334774,-546.22656,104.266120,16.771282,2.452711,4.185177,5.208492,0.743853,-16.445913,-9.057341,5.203734,-4.611980,-9.243179,-2.910447
4,4,0,1692.338682,1627.176669,3296.535326,-531.99520,128.998030,14.050061,7.137925,6.429101,6.554248,2.741421,-24.812666,-13.605760,7.323435,-4.648766,-6.041638,-3.786564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,1604.673859,1499.800469,3185.582386,-375.62567,109.994700,-1.103966,3.337362,5.023191,-12.548165,-8.006364,-5.329917,-0.575209,0.847520,-8.430272,-10.862280,-13.539113
9996,9996,0,1512.619845,1456.740810,3009.677156,-376.19860,133.051180,-12.700054,-2.880110,2.980373,-14.621652,-6.111285,-8.343706,-4.435156,4.592192,-1.841004,-5.927738,-13.076294
9997,9997,0,1652.647172,1438.211296,3096.038277,-377.63596,119.675964,-5.690570,5.225908,7.438576,-12.451978,-7.420503,-9.585105,-4.131483,3.888522,-4.255625,-8.398252,-13.990422
9998,9998,0,1579.832533,1403.813838,3014.873798,-354.54654,120.235430,-5.376558,-5.086518,5.066173,-10.040578,-8.872187,-5.993078,-1.415015,1.019012,-4.854206,-8.119527,-13.371272


In [61]:
from sklearn import svm

X = mfcc_features.drop(['Unnamed: 0','accent'], axis=1)[:5000]
Y = mfcc_features['accent'][:5000]

clf = 

#clf = svm.SVC(probability = True, gamma='auto')
#clf.fit(X, Y)

SVC(gamma='auto', probability=True)

In [ ]:
mfcc_features.drop('accent')

In [172]:
train_csv['accents'][:1000].value_counts()

México             277
Andino             218
España             186
Rioplatense        135
América central    112
Chileno             45
Caribe              27
Name: accents, dtype: int64

In [12]:
#this function is used to extract audio frequency features
def feature_extraction(ds_path, filename, sampling_rate=48000, mfcc_num=20):
    path = "{}{}".format(ds_path, filename)
    features = list()
    audio, _ = librosa.load(path, sr=sampling_rate)
    
    #accents = train_csv[train_csv['path'] == filename].accents.values[0]
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sampling_rate))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=audio, sr=sampling_rate))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sampling_rate))
    #features.append(accents)
    features.append(spectral_centroid)
    features.append(spectral_bandwidth)
    features.append(spectral_rolloff)
    
    # Extract mfcc features. First experiment
    # hop_length=10
    mfcc = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=mfcc_num) # it returns 20 by default.
    for el in mfcc:
        features.append(np.mean(el))
    
    return features
    
        
#features = feature_extraction(train_csv['path'][15])
#print("features: ", features)


In [107]:
classes= ['América central', 'Andino', 'Caribe', 'Chileno', 'España', 'México',
'Rioplatense']

# latinamericano 1
# espana 2

In [76]:
train_csv['path'][1]

'common_voice_es_19749309.wav'

In [33]:
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
  
# Sampling frequency
freq = 44100
  
# Recording duration
duration = 5
  
# Start recorder with the given values 
# of duration and sample frequency
recording = sd.rec(int(duration * freq), 
                   samplerate=freq, channels=2)
  
# Record audio for the given number of seconds
sd.wait()
  
# This will convert the NumPy array to an audio
# file with the given sampling frequency
write("recording0.wav", freq, recording)
  
# Convert the NumPy array to audio file
wv.write("recording1.wav", recording, freq, sampwidth=2)

In [34]:
file_path = "./"
#file_name = train_csv['path'][970]
file_name = 'recording0.wav'
features = feature_extraction(ds_path = file_path, filename = file_name, mfcc_num = 13)

In [100]:
train_csv[train_csv['accents']=='Andino'].iloc[1000]

Unnamed: 0                                                17586
client_id     45e81a8f415d858b75eab90a89f4786639a06db9e2da71...
path                               common_voice_es_20234709.wav
sentence       Estas entidades se pueden dividir en categorías.
up_votes                                                      2
down_votes                                                    0
age                                                       teens
gender                                                     male
accents                                                  Andino
locale                                                       es
segment                                                     NaN
Name: 5550, dtype: object

In [101]:
#test_file = features[1:] # when using train csv file
#mfcc_features.iloc[129899]
test_file = mfcc_features.iloc[5550]

In [102]:
train_csv.iloc[5550]

Unnamed: 0                                                17586
client_id     45e81a8f415d858b75eab90a89f4786639a06db9e2da71...
path                               common_voice_es_20234709.wav
sentence       Estas entidades se pueden dividir en categorías.
up_votes                                                      2
down_votes                                                    0
age                                                       teens
gender                                                     male
accents                                                  Andino
locale                                                       es
segment                                                     NaN
Name: 5550, dtype: object

In [103]:
test_file = test_file.drop(['Unnamed: 0','accent'])

In [104]:
test_file

spectral_centroid     2126.055798
spectral_bandwidth    1432.311014
spectral_rolloff      3617.024059
mfcc1                 -258.400180
mfcc2                  181.968750
mfcc3                  -79.777190
mfcc4                   -7.084609
mfcc5                   29.988040
mfcc6                  -19.086388
mfcc7                    4.147798
mfcc8                    4.508232
mfcc9                   -8.835572
mfcc10                  11.619371
mfcc11                   7.091634
mfcc12                   0.926358
mfcc13                   5.221154
Name: 5550, dtype: float64

In [105]:
classes= ['América central', 'Andino', 'Caribe', 'Chileno', 'España', 'México',
'Rioplatense']

result = model.predict_proba([test_file])

In [106]:
result

array([[0.09343347, 0.15489775, 0.12169715, 0.04690984, 0.36577837,
        0.18258019, 0.03470322]])

In [78]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=5, random_state=0)
gmm.fit(x,y)
gmm.means_


array([[ 1.05200590e-01, -1.01638018e+00, -2.31386767e+00,
        -1.39160307e+00, -8.94668254e-01, -1.24928971e+00,
         3.03219466e-01, -9.18485010e-01, -1.09697137e+00,
        -1.58605062e-01, -8.09485689e-01, -7.43264887e-01,
        -2.72052464e-01, -5.61490796e-01, -1.94209802e-01,
        -4.60500503e-01, -4.98285728e-01],
       [ 8.55709124e-01,  1.54653615e-01, -2.65046863e-01,
         2.37722071e-02, -1.32433305e-01,  1.46153670e-01,
        -1.49569020e-01, -4.87673304e-01, -1.31522613e-01,
        -5.38745677e-02, -4.66432710e-01, -2.67204073e-01,
        -5.46360737e-03, -3.20696383e-01, -3.17654828e-01,
        -6.93881604e-02,  1.42861022e-01],
       [ 8.55709124e-01, -1.21401947e+00, -2.61186353e-01,
        -1.06393632e+00, -3.38231275e-02,  2.78590072e-01,
         8.91145132e-01,  1.04705945e+00,  1.25359418e+00,
         8.83181519e-01,  7.20165985e-01,  1.34771016e+00,
         2.58772542e-01, -9.24610201e-02,  1.10582671e+00,
         5.26730767e-01, -3.0

In [107]:

labels = gmm.predict_proba([test_file])

ValueError: Expected the input data X have 17 features, but got 16 features

In [104]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [106]:
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

TypeError: '(slice(None, None, None), 0)' is an invalid key